# General preparations

In [1]:
from misc.misc import run_in_thread_simple
%load_ext autoreload
%autoreload 2
# %matplotlib notebook
import matplotlib
matplotlib.use('TKAgg')
import os 
import datetime
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
import time 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_09_QKD_random_phases'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

In [3]:
# MPLC
from pianoq.lab.mplc.mplc_device import MPLCDevice
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

Got MPLC


In [4]:
# get masks
import glob 
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
paths = glob.glob(fr'{dir_path}\*.masks')
assert len(paths) == 1, 'there should be one .masks file'
masks_path = paths[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
orig_masks = msks.real_masks

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

## Checklist
- [X] make list
- [X] create masks WFM
- [X] measure single counts 
- [X] mark spots locations 
- [ ] find phases
- [ ] measure coincidence no phases
- [ ] measure with good phases 
- [ ] measure with random phases X 5 

# Create masks WFM

In [3]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path = fr'{dir_path}\{timestamp}_QKD_MUB2.masks'
mplc_sim = create_WFM_QKD_masks(N_iterations=15, out_path=masks_path)

100%|██████████| 30/30 [06:38<00:00, 13.29s/it]


In [4]:
mplc_sim.res._calc_normalized_overlap()
display(np.angle(np.diag(mplc_sim.res.forward_overlap)))
display(np.diag(mplc_sim.res.forward_overlap))
mplc_sim.res._calc_fidelity()
display(mplc_sim.res.fidelity)

array([-0.01440468, -0.01427223, -0.01166631,  0.00382477, -0.00038664,
        0.00150615,  0.02001652,  0.02083826, -0.02170043, -0.00085186],
      dtype=float32)

array([0.62315977-0.00897704j, 0.6437228 -0.00918799j,
       0.6325797 -0.00738021j, 0.6175512 +0.002362j  ,
       0.62247694-0.00024067j, 0.5763015 +0.000868j  ,
       0.60372466+0.01208608j, 0.583443  +0.01215969j,
       0.5951928 -0.01291797j, 0.5889042 -0.00050167j], dtype=complex64)

0.9288585368232875

In [5]:
mplc_sim.res.show_overlap()

In [7]:
fig, axes = plt.subplots(2, 5)
for i, ax in enumerate(axes.flat):
    # ax.imshow(np.angle(res.masks[i][360:720, 140:280]), cmap='gray')
    ax.imshow(np.angle(mplc_sim.res.masks[i][360:720, 140:280]), cmap='gray')

In [6]:
mplc_sim.res.show_all(7)

# Find locations with single counts 

In [5]:
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult

masks = orig_masks.copy()
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

In [6]:
resolution = 1
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal_res_{resolution}.scan'
singles_signal_path = fr'{dir_path}\{timestamp}_QKD_MUB2_idler_res_{resolution}.scan'

## perform scan

In [7]:
t = run_in_thread(idler_scan, integration_time=1.0, coin_window=2e-9, resolution=resolution, out_path=singles_idler_path, half_scan=True)

In [8]:
t = run_in_thread(signal_scan, integration_time=1.0, coin_window=2e-9, resolution=resolution, out_path=singles_signal_path, half_scan=True)

## mark spots locations  

In [14]:
from pianoq_results.scan_result import ScanResult

path_sig = glob.glob(fr'{dir_path}\*signal_res_1.scan')[0]
path_idl = glob.glob(fr'{dir_path}\*idler_res_1.scan')[0]

res_sig = ScanResult()
res_idl = ScanResult()
res_idl.loadfrom(path_idl)
res_sig.loadfrom(path_sig)
res_idl.show_singles()
res_sig.show_singles()

In [ ]:
_ = res_sig.get_xys(1, saveto_dir=dir_path)
_ = res_idl.get_xys(2, saveto_dir=dir_path)

# Find phases

In [7]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder
import glob 
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import time 


locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

masks = orig_masks.copy()
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)

mplc.load_masks(masks, linear_tilts=True)
initiail_phases = np.zeros(25 * 2)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal.phases'

# i,j of corr matrix that is supposed to be strong 
i = 0
j = 0
pf = PhaseFinder(mplc=mplc, integration_time=30, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=initiail_phases, coin_window=2e-9)
pf.m_idl_x.move_absolute(locs_idl[i, 0])
pf.m_idl_y.move_absolute(locs_idl[i, 1])
pf.m_sig_x.move_absolute(locs_sig[j, 0])
pf.m_sig_y.move_absolute(locs_sig[j, 1])

Got Zaber motors!
Got Thorlabs motors!
Got TimeTagger!


In [9]:
initiail_phases = np.zeros(25 * 2)
pf.res.phases = initiail_phases
pf.res.initial_phases = initiail_phases

mplc.restore_location()

In [ ]:
pf.find_phases()

In [6]:
pf.close()

In [ ]:
pf.res.phases

In [ ]:
pf.res.plot_best_phases()

3


# Scan correlations 

In [5]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.mplc_device import MPLCDevice
import glob
import time 

phases_result = PhaseFinderResult()
phases_path = glob.glob(fr'{dir_path}\*.phases')[0]
phases_result.loadfrom(phases_path)

locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']


backlash = 0.0
wait_after_move = 0.3
integration_time = 50
coin_window = 0.4e-9
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_type = 'XXX'
scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s.dscan'
dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window)
time.sleep(3)

if True:
    phases_type = 'good'
    phases = phases_result.phases
    
    masks = orig_masks.copy()
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    masks = add_phase_input_spots(masks, phases)
    
    mplc.load_masks(masks, linear_tilts=True)
    mplc.restore_location()
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
    np.savez(used_phases_path, phase=phases)
    scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s_coin_{int(coin_window*1e12)}.dscan'
    dps.res.path = scan_path
    dps.scan()

if True:
    phases_type = 'none'
    phases = np.zeros_like(phases_result.phases)
    
    masks = orig_masks.copy()
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    masks = add_phase_input_spots(masks, phases)
    
    mplc.load_masks(masks, linear_tilts=True)
    mplc.restore_location()
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
    np.savez(used_phases_path, phase=phases)
    scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s_coin_{int(coin_window*1e12)}.dscan'
    dps.res.path = scan_path
    dps.scan()

if True:
    phases_type = 'random'
    phases = np.random.uniform(0, 2*np.pi, len(phases_result.phases))
    
    
    for i in range(10):
        # several different random phases 
        masks = orig_masks.copy()
        masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
        # phases = np.random.uniform(0, 2*np.pi, len(phases_result.phases))  # Always random right now <-----------------------------------
        masks = add_phase_input_spots(masks, phases)
        
        mplc.load_masks(masks, linear_tilts=True)
        mplc.restore_location()
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
        np.savez(used_phases_path, phase=phases)
        scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s_coin_{int(coin_window*1e12)}.dscan'
        dps.res.path = scan_path
        dps.scan()

Got Zaber motors!
Got Thorlabs motors!
Got TimeTagger!
beginning scan
0, 0: 5970.12, 6339.22, 2.42
0, 1: 5970.44, 5779.40, 0.12
0, 2: 5994.76, 5841.92, 0.10
0, 3: 6005.92, 7508.60, 0.14
0, 4: 5972.62, 6496.04, 0.18
1, 0: 7160.06, 6338.16, 0.08
1, 1: 7173.78, 5777.96, 2.78
1, 2: 7168.12, 5869.78, 0.16
1, 3: 7163.94, 7528.86, 0.28
1, 4: 7172.44, 6497.36, 0.12
2, 0: 7319.76, 6325.96, 0.12
2, 1: 7303.48, 5752.24, 0.16
2, 2: 7298.10, 5859.66, 3.56
2, 3: 7312.68, 7505.86, 0.44
2, 4: 7338.48, 6483.22, 0.06
3, 0: 7960.86, 6317.10, 0.18
3, 1: 7956.98, 5772.98, 0.18
3, 2: 7936.16, 5866.28, 0.24
3, 3: 7959.00, 7528.40, 5.36
3, 4: 7956.58, 6495.88, 0.14
4, 0: 7017.48, 6341.20, 0.22
4, 1: 7000.66, 5750.86, 0.10
4, 2: 6990.64, 5842.06, 0.02
4, 3: 7015.70, 7529.40, 0.20
4, 4: 7015.78, 6465.42, 3.16
beginning scan
0, 0: 5903.26, 6355.00, 1.00
0, 1: 5909.18, 5766.80, 0.60
0, 2: 5914.78, 5858.22, 0.16
0, 3: 5933.14, 7565.10, 0.36
0, 4: 5917.98, 6532.06, 0.48
1, 0: 7091.26, 6334.46, 0.56
1, 1: 7090.74, 5

In [6]:
used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
np.savez(used_phases_path, phase=phases)
dps.scan()

beginning scan
0, 0: 6410.28, 5122.56, 1.94
0, 1: 6410.62, 4742.22, 0.58
0, 2: 6388.90, 4903.08, 0.44
0, 3: 6405.98, 5998.72, 1.28
0, 4: 6392.06, 5643.90, 0.84
1, 0: 7697.28, 5141.18, 0.90
1, 1: 7697.64, 4741.96, 2.28
1, 2: 7690.76, 4937.38, 0.50
1, 3: 7705.68, 6005.56, 0.94
1, 4: 7705.68, 5643.56, 1.16
2, 0: 7531.06, 5150.94, 1.28
2, 1: 7507.12, 4737.52, 0.72
2, 2: 7511.82, 4895.14, 2.80
2, 3: 7490.34, 6002.22, 0.84
2, 4: 7485.66, 5637.12, 1.10
3, 0: 7982.22, 5128.68, 0.82
3, 1: 7998.08, 4729.84, 1.60
3, 2: 7965.64, 4891.74, 0.94
3, 3: 8005.06, 5982.88, 3.00
3, 4: 7974.58, 5612.42, 1.24
4, 0: 7168.86, 5125.40, 0.42
4, 1: 7151.06, 4710.30, 0.30
4, 2: 7147.98, 4896.40, 1.18
4, 3: 7160.88, 5998.80, 1.32
4, 4: 7182.10, 5642.08, 2.36


In [7]:
dps.close()

In [10]:
dps.res.show()
dps.res.show_singles()

In [ ]:
mplc.close()